In [1]:
import re
import json
import logging
import asyncio
from pathlib import Path
from PyPDF2 import PdfReader
from market_agents.memory.config import AgentStorageConfig, load_config_from_yaml
from market_agents.memory.embedding import MemoryEmbedder
from market_agents.memory.knowledge_base_agent import KnowledgeBaseAgent
from market_agents.agents.market_agent import MarketAgent
from market_agents.inference.message_models import LLMConfig
from market_agents.environments.mechanisms.chat import ChatEnvironment
from market_agents.memory.agent_storage.agent_storage_api_utils import AgentStorageAPIUtils
from market_agents.memory.knowledge_base import MarketKnowledgeBase


In [2]:
config_path = Path("../market_agents/memory/memory_config.yaml")
config = load_config_from_yaml(str(config_path))

# Initialize embedding service and storage API utils
embedder = MemoryEmbedder(config)
storage_utils = AgentStorageAPIUtils(
    config=config,
    logger=logging.getLogger("storage_api")
)

hamlet_kb = MarketKnowledgeBase(
    config=config,
    table_prefix="hamlet_kb"
)
await hamlet_kb.initialize()

kb_agent = KnowledgeBaseAgent(market_kb=hamlet_kb)


INFO:market_agents.memory.agent_storage.agent_storage_api_utils:Initializing Agent Storage API Utils with URL: http://localhost:8001


In [4]:
def document_reader(file_path: str) -> str:
    """
    Read content from PDF or TXT files and return cleaned text
    
    Args:
        file_path (str): Path to the document file
        
    Returns:
        str: Cleaned text content from the document
    """
    text_content = ""
    
    if file_path.endswith('.pdf'):
        pdf_reader = PdfReader(file_path)
        for page in pdf_reader.pages:
            page_text = page.extract_text()
            cleaned_text = re.sub(r'FTLN \d+', '', page_text)
            cleaned_text = re.sub(r'^\d+\s*', '', cleaned_text, flags=re.MULTILINE)
            text_content += cleaned_text
    
    elif file_path.endswith('.txt'):
        with open(file_path, 'r', encoding='utf-8') as f:
            text_content = f.read()
    else:
        raise ValueError("Unsupported file format. Please provide a PDF or TXT file.")
            
    return text_content

# Read document content
file_path = "/Users/interstellarninja/Documents/projects/MarketAgents/examples/hamlet_PDF_FolgerShakespeare.pdf"
text_content = document_reader(file_path)

In [ ]:
# Store in knowledge base with metadata
knowledge_id = await hamlet_kb.ingest_knowledge(
    text=text_content,
    metadata={
        "source": "Hamlet",
        "type": "play",
        "author": "William Shakespeare"
    }
)
print(f"Successfully created Hamlet knowledge base with ID: {knowledge_id}")


In [ ]:
# Test some queries
async def query_hamlet(question: str):
    print(f"\nQuestion: {question}")
    print("-" * 80)
    
    results = await kb_agent.retrieve(
        query=question
    )
    
    for i, chunk in enumerate(results, 1):
        print(f"\nResult {i} (Similarity: {chunk.similarity:.3f})")
        print("Text:", chunk.text)
        print("\nContext:", chunk.context)
        print("-" * 80)

# Try different queries
queries = [
    "What is Hamlet's famous 'To be or not to be' soliloquy about?",
    "How does Ophelia die in the play?",
    "What is the role of the ghost in the play?",
    "Describe the relationship between Hamlet and Horatio"
]

for query in queries:
    await query_hamlet(query)

In [3]:
chat_env = ChatEnvironment(name="hamlet_chat")


hamlet_agent = await MarketAgent.create(
    storage_utils=storage_utils,
    agent_id="hamlet_agent",
    use_llm=True,
    llm_config=LLMConfig(
        client="openai",
        model="gpt-4o",
        temperature=0.5,
        max_tokens=1024
    ),
    knowledge_agent=kb_agent, 
    environments={"chat": chat_env},
)

# Add a test message to the chat
hamlet_agent.task = "What is Hamlet's famous 'To be or not to be' soliloquy about?"
chat_env.mechanism.add_user_message(hamlet_agent.task)

# Generate a response
perception = await hamlet_agent.perceive(environment_name="chat")
print(json.dumps(perception, indent=4))

action = await hamlet_agent.generate_action(environment_name="chat", perception=perception)
print(json.dumps(action, indent=4))

reflection = await hamlet_agent.reflect(environment_name="chat")
print(json.dumps(reflection, indent=4))



Cognitive Memory Results:
Memory 1:
{'cognitive_step': 'action', 'content': '{"agent_id": "agent_123", "action": {"thoughts": [{"reasoning": "The user has asked about the \'To be or not to be\' soliloquy from Hamlet, which is a critical moment in the play. This soliloquy is renowned for its exploration of existential themes, where Hamlet contemplates the nature of existence, life, and death. My strategy involves providing a detailed explanation of these themes and how they relate to the broader narrative of the play."}, {"reasoning": "I will start by explaining the context of the soliloquy within the play. Hamlet is grappling with the recent death of his father and the hasty remarriage of his mother to his uncle, who is now the king. This context sets the stage for Hamlet\'s internal struggle and existential questioning."}, {"reasoning": "Next, I will delve into the soliloquy\'s content, focusing on Hamlet\'s contemplation of life and death. He questions whether it is nobler to endure

In [4]:
# Print all accessible instance attributes of the hamlet_agent
for attr_name, attr_value in hamlet_agent.__dict__.items():
    print(f"{attr_name}: {attr_value}")

id: hamlet_agent
role: agent
persona: None
system: None
task: What is Hamlet's famous 'To be or not to be' soliloquy about?
tools: None
output_format: None
llm_config: client='openai' model='gpt-4o' max_tokens=1024 temperature=0.5 response_format='structured_output' use_cache=True
max_retries: 2
metadata: None
interactions: [{'id': 'hamlet_agent', 'name': 'agent', 'system': {'role': 'system', 'content': 'Role: You are a agent agent. You are a long-horizon task execution agent based on verbal reinforcement learning. The current date and time is 2025-01-28 07:21:45.\n\n'}, 'task': 'Tasks: Your are assigned with following tasks:\nPerceive the current state of the chat environment:\n\n## Environment State\n-   content: What is Hamlet\'s famous \'To be or not to be\' soliloquy about?\n  timestamp: 2025-01-28 07:21:44\n  role: user\n\n## Recent Cognitive Memories\n-   cognitive_step: action\n  content: {"agent_id": "agent_123", "action": {"thoughts": [{"reasoning": "The user has asked about 